Pip Installs

In [ ]:
!pip install torchmetrics
!pip install cleverhans
!pip install quantus
!pip install captum
!pip install ranger-adabelief

Imports

In [ ]:
import torch
from torch import nn
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.nn.utils.prune as prune
from torch.utils.data import random_split
from torch.utils.data import Dataset

import torchvision
from torchvision import datasets
from torchvision import transforms
import pandas as pd
from PIL import Image

from torchmetrics import Accuracy
import torch.optim as optim
from cleverhans.torch.attacks.projected_gradient_descent import (projected_gradient_descent)

import quantus
import captum
from captum.attr import Saliency, IntegratedGradients, NoiseTunnel

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
import copy
import gc
import math

import warnings

import os
from itertools import chain

from collections import Counter
from ranger_adabelief import RangerAdaBelief
from pathlib import Path

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
%run GTSRB/utilities.ipynb
%run GTSRB/model_architectures.ipynb

Exception: File `'utilities.ipynb.py'` not found.

Loading Dataset

In [ ]:
epochs = 100
batch_size = 128
image_size = (32, 32)

normalize = transforms.Normalize(mean=[0.4563, 0.4076, 0.3895], std=[0.2298, 0.2144, 0.2259])
gtsrb_transforms = transforms.Compose([transforms.Resize(image_size),transforms.ToTensor(),normalize])

In [ ]:
#dataset =  (root='./datasets' , download=True, train=True, transform = gtsrb_transforms)
train_path = "GTSRB/datasets/Train"

full_gtsrb_dataset = datasets.ImageFolder(root=train_path, transform=gtsrb_transforms)

train_size = int(0.8 * len(full_gtsrb_dataset))
test_size = len(full_gtsrb_dataset) - train_size
train_dataset, test_dataset = random_split(full_gtsrb_dataset, [train_size, test_size])

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Testing samples: {len(test_dataset)}")

Training samples: 31367
Testing samples: 7842


In [ ]:
# Initialize labels to numbers
classes, class_names = class_formation()

Train Model

In [ ]:
def train_adv(model, epsilon, epochs):
    model.train()
    eps = epsilon
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            x_batch = projected_gradient_descent(model, x_batch, eps, eps/10, 40, np.inf)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluating Model after each epoch
        if epochs%10==0:
            prediction, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(prediction.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.4f}")
    return model

In [ ]:
# Setting hyperparameters
actual_model = vgg16()
learning_rate = 1e-4
epochs = 40
eps = 0.1
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(actual_model.parameters(), lr=learning_rate)

In [ ]:
# Actual training of model
model_adversarial = train_adv(model=actual_model.to(device),
                    epsilon = eps,
                    epochs=epochs)

In [ ]:
model_adversarial.to(device)
model_adversarial.eval()

predictions, labels = evaluate_model(model_adversarial, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
print(f"VGG-16 Normal Model test accuracy: {(100 * test_acc):.2f}%")

In [ ]:
# Save model
model_path = Path("GTSRB/models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "vgg_adversarial.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_adversarial.state_dict(), f=model_save_path)